# Data Aggregation and Group Operations

## 10.1 GroupBy Mechanics

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.arange(5),
                   'data2' : np.arange(5)})
df

,key1,key2,data1,data2
0,a,one,0,0
1,a,two,1,1
2,b,one,2,2
3,b,two,3,3
4,a,one,4,4


In [11]:
grouped = df['data1'].groupby(df['key1'])
# 按key1进行分组，并计算data1列的平均值
grouped

grouped变量现在是一个GroupBy对象，除了一些关于分组键df['key1']的一些中间数据之外，它实际上还没有任何操作

In [12]:
grouped.mean() # 求平均值

key1
a    1.666667
b    2.500000
Name: data1, dtype: float64

数据(一个Series)根据分组键进行了聚合，并产生了一个新的Series

In [13]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     2
      two     1
b     one     2
      two     3
Name: data1, dtype: int32

In [14]:
means.unstack()
# 拆堆 将行中的数据透视到列

key2,one,two
key1,,
a,2,1
b,2,3


In [15]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    1.0
            2006    2.0
Ohio        2005    1.5
            2006    4.0
Name: data1, dtype: float64

In [16]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,1.666667,1.666667
b,2.500000,2.500000


In [17]:
df.groupby(['key1', 'key2']).mean() # 对key1，key2同时求平均

data1  data2
key1 key2              
a    one       2      2
     two       1      1
b    one       2      2
     two       3      3

默认情况下，所有的数值列都可以聚合

如果不在意使用Groupby的目的，通用的Groupby方法是size,size方法返回的是包含数组大小信息的Series

In [18]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

#### 注意：分组键中的任何缺失值都将被排除在结果之外

### 10.1.1 Iterating Over Groups

GroupBy对象支持迭代，会生成一个包含组名和数据块的2维元组序列

In [19]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2  data1  data2
0    a  one      0      0
1    a  two      1      1
4    a  one      4      4
b
  key1 key2  data1  data2
2    b  one      2      2
3    b  two      3      3


In [20]:
# 在多个分组键的情况下，元组中的第一个元素是键值的元组
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2  data1  data2
0    a  one      0      0
4    a  one      4      4
('a', 'two')
  key1 key2  data1  data2
1    a  two      1      1
('b', 'one')
  key1 key2  data1  data2
2    b  one      2      2
('b', 'two')
  key1 key2  data1  data2
3    b  two      3      3


In [21]:
pieces = dict(list(df.groupby('key1'))) # 保留索引
pieces['b']

,key1,key2,data1,data2
2,b,one,2,2
3,b,two,3,3


默认情况下，groupby在axis=0的轴向上分组

In [22]:
df.dtypes # 将每一列的数据类型输出

key1     object
key2     object
data1     int32
data2     int32
dtype: object

In [23]:
grouped = df.groupby(df.dtypes, axis=1)
for dtype,group in grouped:
    print(dtype)
    print(group)

int32
   data1  data2
0      0      0
1      1      1
2      2      2
3      3      3
4      4      4
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 10.1.2 Selecting a Column or Subset of Columns

将DataFrame创建的Groupby对象用列名称或列名称数组进行索引搜索时，会产生聚合的列子集的效果

In [24]:
df.groupby(['key1','key2'])[['data2']].mean() # 计算data2列的均值，并获得DataFrame形式的效果

data2
key1 key2       
a    one       2
     two       1
b    one       2
     two       3

#### 如果传递的是列表或数组，则此索引操作返回的对象是分组的DataFrame，如果只有单个列名作为标量传递，则为分组的Series

In [25]:
s_groupby=df.groupby(['key1','key2'])['data2']
s_groupby

In [26]:
s_groupby.mean()

key1  key2
a     one     2
      two     1
b     one     2
      two     3
Name: data2, dtype: int32

### 10.1.3 Grouping with Dicts and Series

In [27]:
people = pd.DataFrame(np.arange(25).reshape(5,5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
# [2,3) 即索引为2的行，[1,2] 索引为1，2的列，将其赋值为NaN
people

,a,b,c,d,e
Joe,0,1.0,2.0,3,4
Steve,5,6.0,7.0,8,9
Wes,10,NaN,NaN,13,14
Jim,15,16.0,17.0,18,19
Travis,20,21.0,22.0,23,24


In [28]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}
by_column = people.groupby(mapping,axis=1)
by_column

In [29]:
by_column.sum()

,blue,red
Joe,5.0,5.0
Steve,15.0,20.0
Wes,13.0,24.0
Jim,35.0,50.0
Travis,45.0,65.0


Series也有相同的功能，可以视为固定大小的映射

In [30]:
map_series=pd.Series(mapping)
map_series # 变成Series形式进行输出

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [31]:
people.groupby(map_series,axis=1).count()  # 每行颜色出现的次数

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 10.1.4 Grouping with Functions

作为分组键传递的函数将会按照每个索引值调用一次，同时返回值会被用作分组名称

In [32]:
people

,a,b,c,d,e
Joe,0,1.0,2.0,3,4
Steve,5,6.0,7.0,8,9
Wes,10,NaN,NaN,13,14
Jim,15,16.0,17.0,18,19
Travis,20,21.0,22.0,23,24


In [33]:
people.groupby(len).sum() # 非缺失值累加，缺失值会自动忽略
# 3---> Joe,Wes,Jim
# 5--->Steve
# 6--->Travis

,a,b,c,d,e
3,25,17.0,19.0,34,37
5,5,6.0,7.0,8,9
6,20,21.0,22.0,23,24


In [34]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a     b     c   d   e
3 one   0   1.0   2.0   3   4
  two  15  16.0  17.0  18  19
5 one   5   6.0   7.0   8   9
6 two  20  21.0  22.0  23  24

### 10.1.5 Grouping by Index Levels

分层索引的数据集有一个非常方便的地方，就是能够在轴索引的某个层级上进行聚合

In [35]:
columns=pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],
                                  [1,3,5,1,3]],
                                 names=['cty','tenor'])
hier_df=pd.DataFrame(np.arange(20).reshape(4,5),columns=columns)
hier_df

cty    US          JP    
tenor   1   3   5   1   3
0       0   1   2   3   4
1       5   6   7   8   9
2      10  11  12  13  14
3      15  16  17  18  19

根据层级分组时，将层级数值或层级名称传递给level关键字

In [36]:
hier_df.groupby(level='cty',axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 Data Aggregation
聚合是指所有根据数组产生标量值的数据转换过程

|函数名|描述|
|------|----|
|count|分组中的非NA值数量|
|sum|非NA值的累加|
|mean|非NA值的均值|
|median|非NA值的算术中位数|
|std，var|无偏的(n-1分母)标准差和方差|
|min，max|非NA值的最小值，最大值|
|prod|非NA值的乘积|
|first，last|非NA值的第一个和最后一个值|

In [37]:
import pandas as pd
import numpy as np

In [38]:
a=pd.Series([-0.204708,0.478943,1.965781])
a.quantile(0.9)

1.6684134

In [39]:
#(3-1)*0.9 ---> 1.8  1和0.8  1表示索引  3代表数组长度
c=a[1]+(a[2]-a[1])*0.8# --->1.668413
c

1.6684134

In [40]:
b=pd.Series([-0.555730,-0.519439])
b.quantile(0.9)
# (2-1)*0.9  --->  0.9
# g=b[0]+(b[1]-b[0])*0.9

-0.5230680999999999

In [41]:
df

,key1,key2,data1,data2
0,a,one,0,0
1,a,two,1,1
2,b,one,2,2
3,b,two,3,3
4,a,one,4,4


In [42]:
grouped=df.groupby('key1') # 按照key1进行分类
grouped

In [43]:
grouped['data1'].quantile(0.9) # 对data1这列数求一个90%的分位数

key1
a    3.4
b    2.9
Name: data1, dtype: float64

使用自己的聚合函数，需要将函数传递给aggregate或agg方法

In [44]:
def peak_to_peak(arr):
    return arr.max()-arr.min()
grouped.agg(peak_to_peak)
# grouped=df.groupby('key1') # 按照key1进行分类

,data1,data2
key1,,
a,4,4
b,1,1


In [45]:
grouped.describe()

data1                                                data2            \
     count      mean       std  min   25%  50%   75%  max count      mean   
key1                                                                        
a      3.0  1.666667  2.081666  0.0  0.50  1.0  2.50  4.0   3.0  1.666667   
b      2.0  2.500000  0.707107  2.0  2.25  2.5  2.75  3.0   2.0  2.500000   

                                           
           std  min   25%  50%   75%  max  
key1                                       
a     2.081666  0.0  0.50  1.0  2.50  4.0  
b     0.707107  2.0  2.25  2.5  2.75  3.0

### 10.2.1 Column-Wise and Multiple Function Application

In [46]:
tips = pd.read_csv('examples/tips.csv')
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [47]:
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill'] # 小费/总账单
tips[:6] # 读取前6行

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


对Sereis或DataFrame所有列进行聚合就是使用aggregate和所需函数，或是调用像mean或std方法

In [48]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct'] 
grouped_pct.agg('mean') # 求均值

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

如果传递的是函数或者函数名的列表，会得到一个列名是这些函数名的DataFrame

In [49]:
grouped_pct.agg(['mean','std',peak_to_peak]) # peak_to_peak为arr.max()-arr.min()
# mean() 为均值
# std()  为标准差

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

若传递的是(name,function)元组的列表，每个元组的第一个元素将作为DataFrame的列名

In [51]:
grouped_pct.agg([('foo','mean'),('bar',np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

可以指定应用到所有列上的函数列表或每一列上要应用的不同函数

In [56]:
functions=['count','mean','max']
result=grouped['tip_pct','total_bill'].agg(functions)
result
# 产生的DataFrame拥有分层列，与分别聚合每一列，再以列名作为keys参数使用concat将结果拼接在一起的结果相同

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [57]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [59]:
# 每个元组的第一个元素将作为DataFrame的列名
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)] # 元组的第一个 元素是函数功能的名称，并作为列名
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

将不同的函数应用到一个或多个列上，需传递含有列名与函数对应关系的字典给agg

In [60]:
grouped.agg({'tip':np.max,'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [61]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### 10.2.2 Returning Aggregated Data Without Row Indexes
向groupby传递as_index=False来禁用分组键作为索引的行为

In [63]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [64]:
tips.groupby(['day','smoker'],as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [66]:
tips.groupby(['day','smoker'],as_index=True).mean()

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

## 10.3 Apply: General split-apply-combine

apply将对象拆分为多块，然后在每一块上调用传递的函数，之后尝试将每一块拼接到一起

In [69]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:] # 升序排列并进行切片处理
top(tips, n=6) # 读取排序中最大的6位

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [77]:
tips.groupby('smoker').apply(top) # 按照smoker进行分组，之后调用apply
# top函数在DataFrame的每一行分组上被调用，之后用panda.concat将函数结果粘贴在一起，并使用分组名作为各组的标签

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [76]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [79]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')
# 每一天中最大的总账单

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [80]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [82]:
result.unstack('smoker') # 拆堆处理

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

### 10.3.1 Suppressing the Group Keys

可以看到所得到的对象具有分组键所形成的分层索引以及每个原始对象的索引，可向groupby传递group_keys=False禁用

In [85]:
tips.groupby('smoker', group_keys=False).apply(top) # 输出前五个最大值

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [86]:
tips.groupby('smoker', group_keys=True).apply(top) # 输出前五个最大值

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

### 10.3.2 Quantile and Bucket Analysis

In [87]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)}) # 1000个随机正态分布数
quartiles = pd.cut(frame.data1, 4) # 根据data1中的最大值与最小值等分成4份（分箱），保证4个区间相等
quartiles[:10]

0    (-2.701, -1.305]
1      (0.085, 1.475]
2    (-2.701, -1.305]
3    (-2.701, -1.305]
4    (-2.701, -1.305]
5     (-1.305, 0.085]
6    (-2.701, -1.305]
7     (-1.305, 0.085]
8     (-1.305, 0.085]
9      (0.085, 1.475]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.701, -1.305] < (-1.305, 0.085] < (0.085, 1.475] < (1.475, 2.865]]

cut返回的Categorical对象可以直接传递给groupby

In [89]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
grouped = frame.data2.groupby(quartiles) # 根据data1的分箱操作进行分组
grouped.apply(get_stats).unstack() # 拆堆处理
# 分箱的形式是data1，数据为data2

,count,max,mean,min
data1,,,,
"(-2.701, -1.305]",89.0,2.809427,-0.095631,-3.318208
"(-1.305, 0.085]",456.0,2.808001,-0.092681,-3.115601
"(0.085, 1.475]",399.0,2.922364,0.061718,-2.810897
"(1.475, 2.865]",56.0,2.828350,0.045062,-2.144519


In [94]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False) # 数据量等长，区间不一定等长
# 将1000个数据分为10组
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack() # 拆堆操作

,count,max,mean,min
data1,,,,
0,100.0,2.809427,-0.085721,-3.318208
1,100.0,2.808001,-0.045350,-3.115601
2,100.0,2.323262,-0.106707,-2.531161
3,100.0,2.246533,-0.182104,-2.351185
4,100.0,2.057650,-0.086038,-1.961082
5,100.0,2.619546,0.080015,-2.089278
6,100.0,2.142902,-0.030858,-2.810897
7,100.0,2.160103,0.113005,-2.472035
8,100.0,2.496838,0.074493,-2.185532


### 10.3.3 Example: Filling Missing Values with Group-Specific       Values

In [97]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan # [::2] ---> [开始：结束：步长]
s

0         NaN
1   -1.176802
2         NaN
3    0.892381
4         NaN
5   -0.018649
dtype: float64

In [98]:
s.fillna(s.mean()) # 用平均值填充缺失值

0   -0.101023
1   -1.176802
2   -0.101023
3    0.892381
4   -0.101023
5   -0.018649
dtype: float64

可填充值按组来变化，一个方法是对数据分组后使用apply和一个在每个数据块上都调用fillna的函数

In [101]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4 # 分组的关键字
# ['East']*4生成了一个包含四个['East']中元素的备份的列表，将列表拼接值在一起
data = pd.Series(np.arange(8), index=states) # 索引为states
data

Ohio          0
New York      1
Vermont       2
Florida       3
Oregon        4
Nevada        5
California    6
Idaho         7
dtype: int32

In [103]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan # 设置缺失值
data

Ohio          0.0
New York      1.0
Vermont       NaN
Florida       3.0
Oregon        4.0
Nevada        NaN
California    6.0
Idaho         NaN
dtype: float64

In [105]:
data.groupby(group_key).mean() # 将East和West自动匹配，分别求平均值

East    1.333333
West    5.000000
dtype: float64

In [106]:
fill_mean = lambda g: g.fillna(g.mean()) # 匿名函数
data.groupby(group_key).apply(fill_mean) # 使用分组的平均值来填充缺失值

Ohio          0.000000
New York      1.000000
Vermont       1.333333
Florida       3.000000
Oregon        4.000000
Nevada        5.000000
California    6.000000
Idaho         5.000000
dtype: float64

In [108]:
fill_values = {'East': 0.5, 'West': -1} #为每个分组预定义了填充值（由于每个分组都有一个内置的name属性）
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio          0.0
New York      1.0
Vermont       0.5
Florida       3.0
Oregon        4.0
Nevada       -1.0
California    6.0
Idaho        -1.0
dtype: float64

### 10.3.4 Example: Random Sampling and Permutation
sample(序列a，n)

功能：从序列a中随机抽取n个元素，并将n个元素生以list形式返回。

In [111]:
# 红桃，黑桃，梅花，方块
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)
deck[:13] #[:13)

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [113]:
def draw(deck,n=5):
    return deck.sample(n)
draw(deck)

6D     6
3C     3
5C     5
7D     7
KD    10
dtype: int64

In [115]:
get_suit = lambda card: card[-1] # -1的位置代表花色，即最后一位，例如1H
deck.groupby(get_suit).apply(draw, n=2)
# 从每个花色中随机抽取两张牌

C  9C      9
   AC      1
D  JD     10
   10D    10
H  10H    10
   AH      1
S  5S      5
   7S      7
dtype: int64

In [117]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)
# 禁用所得到的对象具有分组键所形成的分层索引以及每个原始对象的索引

10C    10
2C      2
KD     10
AD      1
5H      5
AH      1
JS     10
3S      3
dtype: int64

### 10.3.5 Example: Group Weighted Average and Correlation

In [121]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,-0.709004,0.653183
1,a,-1.591297,0.782452
2,a,0.715087,0.910066
3,a,2.494631,0.926182
4,b,0.612870,0.144895
5,b,-0.477809,0.222137
6,b,-0.841362,0.729949
7,b,0.249962,0.446428


In [122]:
# 根据category进行分组加权平均
# 即将各数相乘以相应的单位数，然后加总和求和得到总体值，再除以总的单位数
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights']) # 根据权重参数获得的平均值
grouped.apply(get_wavg)

category
a    0.382971
b   -0.336851
dtype: float64

In [124]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True, # parse_dates尝试将数据解析为datetime，默认为False，如果为True，将尝试解析所有的列
                       index_col=0)
close_px.info()
# info()函数用于打印DataFrame的简要摘要，显示有关的DataFrame的信息，包括索引的数据类型dtype和列的数据类型dtype，非空值的数量和内存的使用

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [125]:
close_px[-4:] # 后4行

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [129]:
spx_corr = lambda x: x.corrwith(x['SPX']) # 计算行与行或者列之间的相关性
rets = close_px.pct_change().dropna()
# dropna 根据每个标签的值是否是缺失值数据来筛选轴标签，并根据允许丢失的数据来确定阈值
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr) # 各列与SPX的相关性

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [131]:
by_year.apply(lambda g:g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### 10.3.6 Example: Group-Wise Linear Regression(不怎么懂)

In [133]:
import statsmodels.api as sm
def regress(data,yvar,xvars):
    Y=data[yvar]
    X=data[xvars]
    X['intercept']=1.
    result=sm.OLS(Y,X).fit()  # 通过OLS回归模型获得各个参数
    return result.params

In [135]:
by_year.apply(regress,'AAPL',['SPX']) # 计算AAPL在SPX回报上的年度线性回归

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 10.4 Pivot Tables and Cross-Tabulation
数据透视表是电子表格程序和其他数据分析软件中常见的数据汇总工具，它根据一个人或多个键聚合一张表的数据，将数据在矩形格式中排列，其中一些分组键是沿着行的，另一些是沿着列的

DataFrame拥有一个pivot_table方法，并且还有一个顶层的pandas.pivot_table函数。除了为groupby提供一个方便的接口，pivot_table还可以添加部分总计，也称作边距

In [136]:
tips.pivot_table(index=['day','smoker']) # 默认是计算分组的平均值

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [139]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker') #  index=['time', 'day'] 为二级索引

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

传递margins=True来扩充这个表来包含部分总计，这会添加ALL行和列标签

In [143]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],columns='smoker',margins=True)
# ALL是均值，且该均值是不考虑吸烟者与非吸烟者（ALL列）或行分组中任何两级的（ALL行） ALL=(No+Yes)/2

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

使用不同的聚合函数，将函数传递给aggfunc

In [145]:
tips.pivot_table('tip_pct',index=['time','smoker'],columns='day',aggfunc=len,margins=True) # 求和

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [147]:
tips.pivot_table('tip_pct',index=['time','size','smoker'],columns='day',aggfunc='mean',fill_value=0) # fill_value将缺失值填充为0

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

|选项名|描述|
|------|----|
|values|需要聚合的列名；默认情况下聚合所有数值型的列|
|index|在结透视表的行上进行分组的列名或其他分组键|
|columns|在结果透视表的列上进行分组的列名或其他分组键|
|aggfunc|聚合函数或函数列表（默认情况下是'mean'），可是groupby上下文的任意有效函数|
|fill_value|在结果中替换缺失值的值|
|dropna|如果为True，将不含所有的条目均为NA的列|
|margins|添加行/列小计和总计（默认为False）|

### 10.4.1 Cross-Tabulations: Crosstab

交叉表(crosstab)是数据透视表的一个特殊情况，计算的是分组中的频率

In [150]:
from io import StringIO
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')

In [151]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [155]:
# 按照国籍和惯用性来总结这些数据，可使用pivot_table,也可pandas.crosstab
pd.crosstab(data.Nationality, data.Handedness, margins=True) # margins=True扩充这个表来包含总计

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


crosstab的前两个参数可以是数组，Series或数组的列表

In [157]:
pd.crosstab([tips.time,tips.day],tips.smoker,margins=True) 
# 以时间和天为标准，统计吸烟者

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244